In [ ]:
#/content/wine.csv

import torch
import torch.nn as nn
import numpy as np
import math
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

class WineData(Dataset):
  def __init__(self):
    #super(WineData,self).__init__()
    xy = np.loadtxt("/content/wine.csv",delimiter=",",dtype=np.float32,skiprows=1)
    self.x = torch.from_numpy(xy[:,1:])
    self.y = torch.from_numpy((xy[:,[0]]).astype(np.int64)).squeeze(1)-1 #[4,1] -> 4 & classes:[1,2,3] -> [0,1,2]
    self.n_samples = xy.shape[0]

  def __getitem__(self,index):
    return self.x[index],self.y[index]

  def __len__(self):
    return self.n_samples

datset = WineData()
dataloader = DataLoader(dataset=datset,batch_size=10,shuffle=True,num_workers=2)

class NeuralNet(nn.Module):
  def __init__(self,in_features,hidden_size,out_features):
    super(NeuralNet,self).__init__()
    self.l1 = nn.Linear(in_features,hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size,out_features)

  def forward(self,x):
    pred = self.l1(x)
    pred = self.relu(pred)
    pred = self.l2(pred)
    return pred

in_features = 13
hidden_size = 20
num_epochs = 2
out_features = 3
model = NeuralNet(in_features,hidden_size,out_features)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

for e in range(num_epochs):
  for smp,lbl in dataloader:
    pred = model(smp)
    l = loss(pred,lbl)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

with torch.no_grad():
  pred = model(torch.tensor([[14.39,1.87,2.45,14.6,96,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290]],dtype=torch.float32))
  _,pred = torch.max(pred,1)
  print(pred+1)

tensor([1])


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset,DataLoader

in_features = 784
hidden_size = 100
out_features = 10
num_epochs = 2
batch_size = 100
l_rt = 0.001

train_dataset = torchvision.datasets.MNIST(root="./data",train=True,transform=transforms.ToTensor(),download=True)
test_dataset = torchvision.datasets.MNIST(root="./data",train=False,transform=transforms.ToTensor())

trn_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
tst_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

dataiter = iter(trn_loader)
smp,lbl = next(dataiter)
print(smp.shape,lbl.shape)

class NeuralNet(nn.Module):
  def __init__(self,in_features,hidden_size,out_features):
    super(NeuralNet,self).__init__()
    self.l1 = nn.Linear(in_features,hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size,out_features)

  def forward(self,x):
    pred = self.l1(x)
    pred = self.relu(pred)
    pred = self.l2(pred)
    return pred

model = NeuralNet(in_features,hidden_size,out_features)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=l_rt)

for e in range(num_epochs):
  for smp,lbl in trn_loader:
    smp = smp.reshape(-1,28*28)
    pred = model(smp)
    l = loss(pred,lbl)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

with torch.no_grad():
  n_cor = 0
  n_sap = 0
  for img,lbl in tst_loader:
    img = img.reshape(-1,28*28)
    pred = model(img)
    _,pred = torch.max(pred.data,1)
    n_sap += lbl.shape[0]
    n_cor += (pred==lbl).sum().item()
  acc = 100.0*n_cor/n_sap
  print(acc)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.2MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 488kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.74MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.24MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

torch.Size([100, 1, 28, 28]) torch.Size([100])
95.26


In [2]:
#Bad CNN only for learning how to setup a CNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

# Hyper-parameters
num_epochs = 1
batch_size = 200
learning_rate = 0.001

# dataset has PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class MyCnn(nn.Module):
  def __init__(self):
    super(MyCnn,self).__init__()
    self.cl1 = nn.Conv2d(3,8,5)
    self.pl1 = nn.MaxPool2d(4,4)
    self.cl2 = nn.Conv2d(8,16,5)
    self.pl2 = nn.MaxPool2d(2,2)
    self.act = nn.ReLU()
    self.act2 = nn.Tanh()
    self.fc1 = nn.Linear(16,100)
    self.fc2 = nn.Linear(100,70)
    self.fc3 = nn.Linear(70,10)

  def forward(self,x):
    pred = self.cl1(x)
    pred = self.act2(pred)
    pred = self.pl1(pred)
    pred = self.cl2(pred)
    pred = self.act2(pred)
    pred = self.pl2(pred)
    pred = pred.reshape(pred.shape[0],-1)
    #print(pred.shape)
    pred = self.fc1(pred)
    pred = self.act(pred)
    pred = self.fc2(pred)
    pred = self.act(pred)
    pred = self.fc3(pred)
    return pred #no softmax because CrossEntropyLoss() automatically does it

model = MyCnn()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for e in range(num_epochs):
  for i,(smp,lbl) in enumerate(train_loader):
    pred = model(smp)
    l = loss(pred,lbl)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    print(f"PIter = {i}, Loss = {l}")

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      n_samples += labels.size(0)
      n_correct += (predicted == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

Files already downloaded and verified
Files already downloaded and verified
PIter = 0, Loss = 2.305555820465088
PIter = 1, Loss = 2.3069732189178467
PIter = 2, Loss = 2.3078315258026123
PIter = 3, Loss = 2.306854009628296
PIter = 4, Loss = 2.3016655445098877
PIter = 5, Loss = 2.299232006072998
PIter = 6, Loss = 2.301478385925293
PIter = 7, Loss = 2.2942872047424316
PIter = 8, Loss = 2.2893900871276855
PIter = 9, Loss = 2.2868826389312744
PIter = 10, Loss = 2.290618896484375
PIter = 11, Loss = 2.2757112979888916
PIter = 12, Loss = 2.278735876083374
PIter = 13, Loss = 2.2676897048950195
PIter = 14, Loss = 2.2665085792541504
PIter = 15, Loss = 2.2573466300964355
PIter = 16, Loss = 2.2531111240386963
PIter = 17, Loss = 2.258697271347046
PIter = 18, Loss = 2.2471930980682373
PIter = 19, Loss = 2.2610340118408203
PIter = 20, Loss = 2.245774269104004
PIter = 21, Loss = 2.2468371391296387
PIter = 22, Loss = 2.2179503440856934
PIter = 23, Loss = 2.2092299461364746
PIter = 24, Loss = 2.219406366